### Initialization

# CONFIGURE HERE:

In [ ]:
"""CONSTANTS""" 

CATALOG_ID = ""
ORIGIN_TRANSFER_USER = ""
ORIGIN_URL = ""
ITEM_ID = "" # INSERT HERE

In [ ]:
"""IMPORTS"""

from arcgis.gis import GIS
from arcgis.gis import Item
from arcgis import __version__
from arcgis.features import FeatureLayerCollection, Table
from arcgis.mapping import WebMap
from datetime import datetime

import pandas as pd
import tempfile

import uuid
import json
import tempfile

from getpass import getpass

In [ ]:

ITEM_COPY_PROPERTIES = ['title', 'type', 'typeKeywords', 'description', 'tags',
                        'snippet', 'extent', 'spatialReference', 'name',
                        'accessInformation', 'licenseInfo', 'culture', 'url']

TEXT_BASED_ITEM_TYPES = frozenset(['Web Map', 'Feature Service', 'Map Service','Web Scene', 'Dashboard',
                                   'Image Service', 'Feature Collection', 
                                   'Feature Collection Template',
                                   'Web Mapping Application', 'Mobile Application', 
                                   'Symbol Set', 'Color Set',
                                   'Windows Viewer Configuration'])

FILE_BASED_ITEM_TYPES = frozenset(['File Geodatabase','CSV', 'Image', 'KML', 'Locator Package',
                                  'Map Document', 'Shapefile', 'Microsoft Word', 'PDF',
                                  'Microsoft Powerpoint', 'Microsoft Excel', 'Layer Package',
                                  'Mobile Map Package', 'Geoprocessing Package', 'Scene Package',
                                  'Tile Package', 'Vector Tile Package'])

RELATIONSHIP_TYPES = frozenset(['Map2Service', 'WMA2Code',
                                'Map2FeatureCollection', 'MobileApp2Code', 'Service2Data',
                                'Service2Service'])

_version = [int(i) for i in __version__.split('.')]


### CONFIGURE HERE

In [ ]:
### HARDCODED CONFIG FOR TESTING SM TRANSFER

origin_pass = getpass(prompt=f"Enter the password for user {ORIGIN_TRANSFER_USER}: ")

# Establish origin and target GIS organizations
print("Connecting ...")
origin = GIS(ORIGIN_URL, ORIGIN_TRANSFER_USER, origin_pass, expiration=9999)
print("Connection Successful.")

destination = GIS("home", expiration=9999)

In [ ]:
catalog = destination.content.get(CATALOG_ID)
catalog = catalog.tables[0]
    

### Helper Functions

In [ ]:
def export_resources(item, save_path=None, file_name=None):
    """
    Helper function, from https://developers.arcgis.com/python/samples/clone-storymap-version2/
    Export's the data's resources as a zip file
    """
    
    url = f'{item._gis._portal.resturl}content/users/{item._user_id}/items/{item.itemid}/resources/export'
    if save_path is None:
        save_path = tempfile.gettempdir()
    if file_name is None:
        file_name = f"{uuid.uuid4().hex[:6]}.zip"
    params = {'f' : 'zip'}
    con = item._gis._portal.con
    resources = con.get(url, params=params,
                        out_folder=save_path,
                        file_name=file_name,
                        try_json=False)
    return resources

def get_layer_item_ids(wm) -> list:
    """
    Helper function from https://developers.arcgis.com/python/guide/cloning-content/
    
    Returns the related items in a webmap.
    
    Params:
        wm (argis.gis.Item): Webmap item to be inspected.
    Returns:
        wm_id_list (list): List of related items in the web map.
    """
    wmo = WebMap(wm)
    wm_id_list = []
    
    for layer in wmo.layers:
        try:
            fsvc = FeatureLayerCollection(layer['url'][:-1], origin)
            if not fsvc.properties['serviceItemId'] in wm_id_list:
                wm_id_list.append(fsvc.properties['serviceItemId'])
        except Exception as e:
            continue
    return wm_id_list

def iterate_all(iterable, returned="key"):
    # Credits: https://gist.github.com/PatrikHlobil/9d045e43fe44df2d5fd8b570f9fd78cc
    
    """Returns an iterator that returns all keys or values
       of a (nested) iterable.
       
       Arguments:
           - iterable: <list> or <dictionary>
           - returned: <string> "key" or "value"
           
       Returns:
           - <iterator>
    """
  
    if isinstance(iterable, dict):
        for key, value in iterable.items():
            if returned == "key":
                yield key
            elif returned == "value":
                if not (isinstance(value, dict) or isinstance(value, list)):
                    yield value
            else:
                raise ValueError("'returned' keyword only accepts 'key' or 'value'.")
            for ret in iterate_all(value, returned=returned):
                yield ret
    elif isinstance(iterable, list):
        for el in iterable:
            for ret in iterate_all(el, returned=returned):
                yield ret

def get_dash_wm(dash) -> list:
    """
    From https://developers.arcgis.com/python/guide/cloning-content/#helper-functions
    
    Returns a list of all Web Maps participating in a Dashboard. 
    
    Arguments:
        dash (item): Dashboard to return participating Web Maps from.
    Returns:
        (list): All Web Maps partipating in the dashboard. 
    
    """
    return [origin.content.get(widget['itemId']) 
            for widget in dash.get_data()["desktopView"]['widgets']
            if widget['type'] == "mapWidget"]


### Transfer Functions

In [ ]:
def wc_transfer(destination: GIS, 
                records: Table = None, 
                items: list = [], 
                logging: bool = False) -> list:
    
    """
    Performs a web content transfer of items from an origin to destination AGOL. 
    
    Arguments:
        destination (arcgis.gis.GIS): Destination GIS for the given items
        items (list): a list of Items to be transferred.
        records (Table): Hosted Table item to memoize transfers to. Must be passed if logging=True
        logging (bool): If True, enables catalog memoization, pushing a transfer record to a Hosted Table in AGOL.  
    """
    
    origin_to_destination_ids = {}
    
    for item in items:
        if item.owner != ORIGIN_TRANSFER_USER:
            item.reassign_to(ORIGIN_TRANSFER_USER)
    
    item_titles = [item.title for item in items]
    
    print("Performing Web Content transfer for the following items: ")
    for title in item_titles:
        print(title)
        
    for item in items:
        try:
            if item.groupDesignations == 'livingatlas' or 'livingatlas' in item.groupDesignations:
                print(f"{item.title} is a Living Atlas item and therefore can only be referenced, not copied. Removing it from transfer.")
                return
            if 'Requires Subscription' in item.typeKeywords:
                print(f"{item.title} is a premium subscription item and therefore can only be referenced, not copied. Removing it from transfer.")
                return
            if 'utility.arcgis.com/usrsvcs' in item.url:
                print(f"{item.title} is a referenced  item and therefore can only be referenced, not copied. Removing it from transfer.")
                return
        except TypeError:
            continue
    
    for item in items:
        if item.id == ITEM_ID:
            destination.content.create_folder(item.title)
                            
    destination_items = destination.content.clone_items(items, folder=item.title)
    
    print("Item(s) cloned successfully. Updating tags ... ")
    now = datetime.now()
    tag = f"src_{origin.properties['urlKey']}_{now.month}/{now.day}/{now.year}-{now.hour}:{now.minute}"
    
    for item in destination_items:
        item.update({'tags': tag})
    for item, destitem in zip(items, destination_items):
        destitem.update({'tags': item.tags})
        
    # build origin to destination map and memoize to catalog
    origin_item_index = 0
    for destination_item in destination_items:            
        origin_to_destination_ids[item.id] = destination_item.id
        
        if item.id == ITEM_ID:
            destination_item.move(destination_item.title) 
                    
        if logging:   
             
            adds = {"attributes":
                {
                    "source_id": items[origin_item_index].id,
                    "destination_id": destination_item.id,
                    "title": destination_item.title,
                    "owner": destination_item.owner,
                    "transfer_date": str(datetime.now())
                }
            }
            
            records.edit_features(adds=[adds])
        
        origin_item_index += 1
        
    print("Web Content Transfer complete.")
    
    for item in destination_items:
        try:
            children = get_layer_item_ids(wm=item)
            for child in children:
                child_item = destination.content.get(child)
                child_item.move(item.title)
        except TypeError:
            continue
    
    return destination_items
    
def dash_transfer(destination: GIS, 
                  dash: Item, 
                  swizzle: bool = True, 
                  records: Table = None, 
                  logging: bool = False) -> None:
    
    """    
    Performs a web content transfer of a dashboard to destination AGOL.
    
    Arguments:
        destination (GIS): Destination GIS for the given Dashboard
        dash (Item): A Dashboard item in the origin GIS
        swizzle (bool): If True, enables JSON swizzling to map keys to values. Future proofing for ArcGIS API 2.2 release.
        records (Table): Hosted Table item to memoize transfers to. Must be passed if logging=True
        logging (bool): If True, enables catalog memoization, pushing a transfer record to a Hosted Table in AGOL.  
    """
        
    dash_elements = get_dash_wm(dash=dash)
    wm_items = {} # origin to destination ids
    
    if dash.owner != ORIGIN_TRANSFER_USER:
        dash.reassign_to(ORIGIN_TRANSFER_USER)
    
    try:     
        if dash.groupDesignations == 'livingatlas':
            print(f"{item.title} is a Living Atlas item and therefore can only be referenced, not copied. Aborting this dash transfer.")
            return
        if 'Requires Subscription' in dash.typeKeywords:
            print(f"{item.title} is a premium subscription item and therefore can only be referenced, not copied. Aborting this dash transfer.")
            return
    except TypeError:
        pass
            
    print(f"Creating destination folder for dashboard {dash.title} ...")
    destination.content.create_folder(dash.title)
    
    for ele in dash_elements:
        
        if ele.owner != ORIGIN_TRANSFER_USER:
            ele.reassign_to(ORIGIN_TRANSFER_USER)
        
        """
        if logging:
            
            records_df = pd.DataFrame.spatial.from_layer(records)
            
            if ele.id in records_df['source_id'].unique():
                try:
                    ele_dest_id  = records_df.loc[records_df['source_id'] == ele.id, 'destination_id'].values[0]
                    wm_items[ele.id] = ele_dest_id
                    destination_item = destination.content.get(ele_dest_id)
                    destination_item.move(dash.title)
                except IndexError:
                    print(f"Failed to transfer {ele.title}. Item may already exist in destination.")
                    continue
        """
                    
        # if the item participating in the dashboard has not yet been cloned: 
        try:
            print(f"Transferring {ele.title} to destination org, moving to Web Content transfer workflow ... ")
            wc = wc_transfer(destination=destination, items=[ele])
            web_map_dest = [item.id for item in wc if item.type == "Web Map"]
            wm_items[ele.id] = web_map_dest[0]
            for item in wc:
                print(f"Moving {item.title} to folder {dash.title} ... ")
                item.move(dash.title)
                
                """
                ### Deprecating, as this is already done recursively within the wc_transfer function... 
                
                if logging:
                    
                    key_list = list(wm_items.keys())
                    val_list = list(wm_items.values())
                    position = val_list.index(destination_item.id)
                    
                    new_record = pd.DataFrame({"source_id": key_list[position],
                        "destination_id": item.id,
                        "title": item.title,
                        "owner": item.owner})
                    records_table.edit_features(adds=new_record)
                """
        
        except IndexError:
            print(f"Item {ele.title} has already been transferred, applying destination-side edits ... ")
            ele_from_search = destination.content.search(query=f"typekeywords:source-{ele.id}")[0]
            wm_items[ele.id] = ele_from_search.id
            ele_from_search.move(dash.title)

    print(f"Participating items handled, transferring dashboard {dash.title} ... ")

    if swizzle == False:
        dest_dash = destination.content.clone_items(items=[dash], item_mapping=wm_items, folder=dash.title)
    else:
        dest_dash = destination.content.clone_items(items=[dash], folder=dash.title)

    
    now = datetime.now()
    tag = f"src_{origin.properties['urlKey']}_{now.month}/{now.day}/{now.year}-{now.hour}:{now.minute}"

    for item in dest_dash:
        item.update({'tags': tag})
        item.update({'tags': dash.tags})
    
    if logging:
        
        adds = {"attributes":
            {
                "source_id": dash.id,
                "destination_id": dest_dash[0].id,
                "title": dest_dash[0].title,
                "owner": dest_dash[0].owner,
                "transfer_date": str(datetime.now())
            }
        }
        
        records.edit_features(adds=[adds])
        
    if swizzle:
        # Swizzle the old and new IDs
        cloned_dash = dest_dash[0]
        dash_json = cloned_dash.get_data()
        dash_str = json.dumps(dash_json)
        
        for key, val in wm_items.items():
            dash_str = dash_str.replace(key, val)
        
        updated_data = json.loads(dash_str)

        cloned_dash.update(item_properties = {}, data = updated_data)

    print(f"Dashboard clone successful. Refresh your content page.")

def sm_transfer(destination: GIS,  
                item: Item,
                records: Table = None, 
                logging : bool = False) -> None:
    """
    Adapted code sample from https://developers.arcgis.com/python/samples/clone-storymap-version2/
    
    Transfer protocol for Story Maps and their web content items. Does not call copy_items() as protocol is different for this content.
    
    Arguments:
        destination (GIS): Destination GIS for the origin item.
        item (Item): Story Map item to transfer from the origin.
        records (Table): Hosted Table item to memoize transfers to. Must be passed if logging=True
        logging (bool): If True, enables catalog memoization, pushing a transfer record to a Hosted Table in AGOL.  

    """
    
    story_map = item
    
    orig_thumbnail = story_map.download_thumbnail()
    
    destination.content.create_folder(story_map.title)    
    
    # check version to apply relevant protocol
    if _version <= [1, 8, 2]:
        resource = export_resources(item=story_map)
    else:
        resource = story_map.resources.export()

    # get story map item data from json to store related maps
    story_map_json = story_map.get_data(try_json=True)

    web_maps = set([v['data']['itemId'] for k, v in story_map_json['resources'].items() \
            if v['type'].lower().find('webmap')>-1])
    express_maps = set([v['data']['itemId'] for k, v in story_map_json['resources'].items() \
            if v['type'].lower().find('expressmap')>-1])


    webmap_mapper = {} # keys are origin IDs, values are destination IDs
    for wm in web_maps:
        webmap_to_copy = origin.content.get(wm)
        
        if webmap_to_copy == None:
            print(f"Webmap Item {wm.title} in Storymap not found in the org. Skipping...")
            continue
        else:    
            
            """
            if logging:
                
                records_table = records.tables[0]
                records_df = pd.DataFrame.spatial.from_layer(records_table)
                
                if webmap_to_copy.id in records_df['source_id'].unique():
                    webmap_destination_id = records_df.loc[records_df['source_id'] == webmap_to_copy.id, 'destination_id']
                    webmap_mapper[webmap_to_copy.id] = webmap_destination_id
            else:
            """
            
            cloned_webmaps = destination.content.clone_items([webmap_to_copy])
            webmap_mapper[webmap_to_copy.id] = [i for i in cloned_webmaps if i.type == 'Web Map'][0].id

            # memoize tranfer to catalog
            if logging:
                
                adds = {"attributes":
                    {
                        "source_id": webmap_to_copy.id,
                        "destination_id": webmap_mapper[webmap_to_copy.id],
                        "title": webmap_to_copy.title,
                        "owner": webmap_to_copy.owner,
                        "type": webmap_to_copy.type,
                        "transfer_date": str(datetime.now())
                    }
                }
                
                records.edit_features(adds=[adds])
                
            for wm in cloned_webmaps:
                wm.move(story_map.title)
        
    # remap the old itemid to the new one
    story_map_text = json.dumps(story_map_json)

    for key, val in webmap_mapper.items():
        story_map_text = story_map_text.replace(key, val)

    new_item = destination.content.add({'type' : story_map.type,
                             'tags' : story_map.tags,
                             'title' : story_map.title,
                             'description' : story_map.description,
                             'typeKeywords' : story_map.typeKeywords,
                             'extent' : story_map.extent,
                             'text' :story_map_text}
                            )

    # bring in the storymap resources exported to a zip archive earlier
    new_item.resources.add(resource, archive=True)

    # update the url
    new_item.update({'url': story_map.url.replace(story_map.id, new_item.id)})
    new_item.update(thumbnail=orig_thumbnail)
    new_item.move(story_map.title)
    
    if logging:
    
        adds = {"attributes":
            {
                "source_id": item.id,
                "destination_id": new_item[0].id,
                "title": new_item[0].title,
                "owner": new_item[0].owner,
                "type": new_item[0].type,
                "transfer_date": str(datetime.now())
            }
        }
        
        records.edit_features(adds=[adds])
    
    print(f"StoryMap transfer complete. You can visit it at {new_item.homepage}")

def transfer():
    """
    Runs the appropriate transfer function for the Item associated with the ITEM_ID global at the top of the notebook.
    """
    
    item_origin = origin.content.get(ITEM_ID)
    
    # reassign item to origin transfer user
    if item_origin.owner != ORIGIN_TRANSFER_USER:
        item_origin.reassign_to(ORIGIN_TRANSFER_USER)
    
    # decide on appropriate workflow for item
    if item_origin.type == "StoryMap":
        sm_transfer(destination=destination, item=item_origin)
    elif item_origin.type == "Dashboard":
        dash_transfer(destination=destination, dash=item_origin, swizzle=True)
    else:
        wc_transfer(destination=destination, items=[item_origin])


In [ ]:
transfer()